# Deep Multi-Pair Forex Trading System - Colab Training

**Phases Implemented:**
- Phase 11 & 12: Godlike Scalper + Machine Gunner
- Phase 13: ADX Filter + Hybrid Reward
- Phase 14: Mid-Price Target (Noise Reduction)
- Phase 15 & 17: Rich Factor Expansion (Mined 50 Features)
- Phase 16 & 18: PER (Priority Tuned to Alpha=0.7)

**Data Location:** `/content/drive/MyDrive/data/{symbol}.csv`

### ⚡ enable GPU
To speed up training, ensure you really are using a GPU Runtime:
1. In the menu, go to **Runtime** > **Change runtime type**.
2. Select **T4 GPU** (or better) under Hardware accelerator.
3. Click **Save**.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Verify GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU NOT Detected! Please enable it in Runtime > Change runtime type.")

In [ ]:
# Install dependencies
!pip install pandas_ta

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import pandas_ta as ta
import numpy as np
from tqdm import tqdm
import os
from torch.utils.data import Dataset, DataLoader
import random
import matplotlib.pyplot as plt

In [ ]:
# --- CONFIGURATION ---
class Settings:
    # --- TRADING PAIRS & PROFILES ---
    # OPTIMIZATION: Zero Commission & Balanced Scaling
    PAIR_CONFIGS = {
        'XAUUSD': {
            'spread': 0.20,
            'commission': 0.0,       # Zero Comm
            'scaling_factor': 5.0,   # Increased High Scaling for Gold (Difficulty Balance)
            'contract_size': 100
        },
        'EURUSD': {
            'spread': 0.0001,
            'commission': 0.0,       # Zero Comm
            'scaling_factor': 10000.0,
            'contract_size': 100000
        },
        'GBPUSD': {
            'spread': 0.0002,
            'commission': 0.0,       # Zero Comm
            'scaling_factor': 10000.0,
            'contract_size': 100000
        }
    }
    PAIRS = list(PAIR_CONFIGS.keys())
    
    # Core
    SEQUENCE_LENGTH = 12 # Scalper: React to last hour only
    
    # RICH FACTORS (Mined Phase 17 - Top 50) + ADX (Filter)
    FEATURES = [
        'spread_close_mid_price', 'rel_close_mid_price', 'spread_low_close', 'rel_low_close', 
        'spread_high_close', 'rel_high_close', 'diff_close_close_lag1', 'rel_open_close', 
        'spread_open_close', 'ratio_close_lag1', 'diff_close_lag1', 'diff_close_mid_price_lag1', 
        'diff_close_low_lag1', 'diff_close_high_lag1', 'ratio_close_lag2', 'diff_close_close_lag2', 
        'diff_close_lag2', 'diff_close_mid_price_lag2', 'diff_close_open_lag1', 'diff_close_open_lag2', 
        'spread_open_high', 'diff_low_close_lag1', 'spread_open_mid_price', 'ratio_close_lag3', 
        'diff_close_low_lag2', 'rel_open_low', 'diff_close_high_lag2', 'diff_close_close_lag3', 
        'rel_open_mid_price', 'spread_open_low', 'rel_open_high', 'diff_mid_price_close_lag1', 
        'ratio_close_lag5', 'diff_close_lag3', 'diff_close_low_lag3', 'diff_close_open_lag3', 
        'ratio_low_lag1', 'diff_close_high_lag3', 'rel_low_mid_price', 'diff_high_close_lag1', 
        'spread_high_low', 'diff_close_high_lag5', 'rel_high_low', 'diff_close_mid_price_lag3', 
        'spread_high_mid_price', 'rel_high_mid_price', 'spread_low_mid_price', 'diff_low_low_lag1', 
        'diff_mid_price_mid_price_lag1', 'diff_mid_price_lag1',
        'adx' # Start index -1 for Filter
    ]
    
    INPUT_DIM = len(FEATURES)
    HIDDEN_DIM = 128
    NUM_LAYERS = 2
    DROPOUT = 0.2
    OUTPUT_DIM = 3 

    # Training
    EPOCHS = 50 
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001 # Aggressive Learning for Binary Scalper
    GAMMA = 0.85  # Scalper: Greedy for immediate reward
    EPSILON_START = 1.0
    EPSILON_DECAY = 0.92
    EPSILON_MIN = 0.01
    
    # PER (Prioritized Experience Replay)
    # PLAN 3 Tuned Value
    PER_ALPHA = 0.7
    PER_BETA_START = 0.4
    
    # Data
    TRAIN_SPLIT_INDEX = 420000
    ATR_PERIOD = 14

In [ ]:
# --- MODEL (Brain) ---
class QNetwork(nn.Module):
    def __init__(self, input_dim=Settings.INPUT_DIM, 
                 hidden_dim=Settings.HIDDEN_DIM, 
                 num_layers=Settings.NUM_LAYERS, 
                 dropout=Settings.DROPOUT, 
                 output_dim=Settings.OUTPUT_DIM):
        super(QNetwork, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_step_out = lstm_out[:, -1, :]
        q_values = self.fc(last_step_out)
        return q_values

In [ ]:
# --- PRIORITIZED EXPERIENCE REPLAY (PER) ---
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.write = 0
        self.count = 0

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        if left >= len(self.tree):
            return idx
        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1
        self.data[self.write] = data
        self.update(idx, p)
        self.write += 1
        if self.write >= self.capacity:
            self.write = 0   
        if self.count < self.capacity:
            self.count += 1

    def update(self, idx, p):
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return (idx, self.tree[idx], self.data[dataIdx])

class PrioritizedReplayBuffer:
    def __init__(self, capacity=10000, alpha=0.7):
        self.tree = SumTree(capacity)
        self.alpha = alpha 
        self.capacity = capacity
        
    def push(self, state, action, reward, next_state, done):
        max_prio = np.max(self.tree.tree[-self.tree.capacity:]) 
        if max_prio == 0:
            max_prio = 1.0
        data = (state, action, reward, next_state, done)
        self.tree.add(max_prio, data)

    def sample(self, batch_size, beta=0.4):
        batch = []
        idxs = []
        segment = self.tree.total() / batch_size
        priorities = []

        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            s = random.uniform(a, b)
            (idx, p, data) = self.tree.get(s)
            if data == 0 or data is None:
                 valid_idx = random.randint(0, self.tree.count - 1)
                 data = self.tree.data[valid_idx]
                 idx = valid_idx + self.capacity - 1
                 p = self.tree.tree[idx]
            priorities.append(p)
            batch.append(data)
            idxs.append(idx)

        sampling_probabilities = np.array(priorities) / self.tree.total()
        is_weights = np.power(self.tree.count * sampling_probabilities, -beta)
        is_weights /= is_weights.max()
        
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.array(states), 
            np.array(actions), 
            np.array(rewards, dtype=np.float32), 
            np.array(next_states), 
            np.array(dones, dtype=bool),
            idxs,
            np.array(is_weights, dtype=np.float32)
        )

    def update_priorities(self, idxs, errors):
        for idx, error in zip(idxs, errors):
            p = (abs(error) + 1e-5) ** self.alpha
            self.tree.update(idx, p)

In [ ]:
# --- DYNAMIC FEATURE GENERATION ---
def prepare_features(df):
    if df.empty:
        return df
    df = df.copy()

    # --- MID-PRICE (Noise Reduction) ---
    df['mid_price'] = (df['high'] + df['low']) / 2.0

    # --- FILTER INDICATORS ---
    bb = ta.bbands(df['mid_price'], length=20, std=2)
    bb.columns = ['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0']
    df = pd.concat([df, bb], axis=1)

    adx_df = ta.adx(df['high'], df['low'], df['close'], length=14)
    df['adx'] = adx_df['ADX_14'] / 100.0
    
    # --- DYNAMIC FEATURE GENERATION ---
    for feature in Settings.FEATURES:
        if feature in df.columns:
            continue
        try:
            parts = feature.split('_')
            type_ = parts[0]
            
            # REL & SPREAD (Already Robust)
            if type_ == 'rel':
                rest = feature.replace('rel_', '')
                known_cols = ['open', 'high', 'low', 'close', 'mid_price']
                c1, c2 = None, None
                for k in known_cols:
                    if rest.startswith(k + '_'):
                        c1 = k
                        c2 = rest.replace(k + '_', '')
                        break
                if c1 and c2:
                    df[feature] = df[c1] / df[c2]
            elif type_ == 'spread':
                rest = feature.replace('spread_', '')
                known_cols = ['open', 'high', 'low', 'close', 'mid_price']
                c1, c2 = None, None
                for k in known_cols:
                    if rest.startswith(k + '_'):
                        c1 = k
                        c2 = rest.replace(k + '_', '')
                        break
                if c1 and c2:
                    df[feature] = df[c1] - df[c2]
            
            # RATIO & DIFF (Robust Unified Logic)
            elif type_ == 'ratio':
                rest = feature.replace('ratio_', '')
                if '_lag' in rest:
                    base, lag_str = rest.split('_lag')
                    lag = int(lag_str)
                    
                    known_cols = ['open', 'high', 'low', 'close', 'mid_price']
                    c1, c2 = None, None
                    
                    if base in known_cols:
                        df[feature] = df[base] / df[base].shift(lag)
                    else:
                        for k in known_cols:
                            if base.startswith(k + '_'):
                                c1 = k
                                c2 = base.replace(k + '_', '')
                                break
                        if c1 and c2:
                            df[feature] = df[c1] / df[c2].shift(lag)
                            
            elif type_ == 'diff':
                 rest = feature.replace('diff_', '')
                 if '_lag' in rest:
                    base, lag_str = rest.split('_lag')
                    lag = int(lag_str)
                    
                    known_cols = ['open', 'high', 'low', 'close', 'mid_price']
                    c1, c2 = None, None
                    
                    if base in known_cols:
                         df[feature] = df[base] - df[base].shift(lag)
                    else:
                         for k in known_cols:
                            if base.startswith(k + '_'):
                                c1 = k
                                c2 = base.replace(k + '_', '')
                                break
                            
                         if c1 and c2:
                            df[feature] = df[c1] - df[c2].shift(lag)
        except:
            pass

    df.fillna(0, inplace=True)
    df = df.iloc[50:] 
    return df

class TradingDataset(Dataset):
    def __init__(self, feature_data, close_prices, seq_len):
        self.feature_data = feature_data
        self.close_prices = close_prices
        self.seq_len = seq_len
        self.valid_indices = range(seq_len, len(feature_data) - 1)

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        i = self.valid_indices[idx]
        state_window = self.feature_data[i - self.seq_len : i]
        next_state_window = self.feature_data[i - self.seq_len + 1 : i + 1]
        curr_price = self.close_prices[i-1]
        next_price = self.close_prices[i]
        return {
            'state': torch.FloatTensor(state_window),
            'next_state': torch.FloatTensor(next_state_window),
            'curr_price': torch.tensor(curr_price, dtype=torch.float32),
            'next_price': torch.tensor(next_price, dtype=torch.float32)
        }

In [ ]:
# --- TRAINING ENGINE (PER EQUIPPED) ---
def train_model(symbol, csv_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training {symbol} on {device}...")
    
    profile = Settings.PAIR_CONFIGS.get(symbol, Settings.PAIR_CONFIGS['XAUUSD'])
    SCALING_FACTOR = profile['scaling_factor']
    SPREAD = profile['spread']
    COMMISSION = profile['commission']
    
    # Load Data
    df = pd.read_csv(csv_path, parse_dates=['time'], index_col='time')
    df = prepare_features(df)
    
    feature_data = df[Settings.FEATURES].values
    close_prices = df['mid_price'].values
    
    dataset = TradingDataset(feature_data, close_prices, Settings.SEQUENCE_LENGTH)
    
    # --- POPULATE PER MEMORY ---
    memory_capacity = len(dataset)
    memory = PrioritizedReplayBuffer(capacity=memory_capacity, alpha=Settings.PER_ALPHA)
    
    print(f"Populating Replay Buffer with {memory_capacity} transitions...")
    temp_loader = DataLoader(dataset, batch_size=4096, shuffle=False)
    
    for batch in tqdm(temp_loader):
        states = batch['state'].numpy()
        next_states = batch['next_state'].numpy()
        curr_prices = batch['curr_price'].numpy()
        next_prices = batch['next_price'].numpy()
        
        # HACK: Store transition details in buffer as 'data' tuple
        for i in range(len(states)):
            memory.push(states[i], next_states[i], curr_prices[i], next_prices[i], False)
    
    policy_net = QNetwork().to(device)
    optimizer = optim.Adam(policy_net.parameters(), lr=Settings.LEARNING_RATE)
    loss_fn = nn.MSELoss(reduction='none') # Important for IS weights
    
    epsilon = Settings.EPSILON_START
    loss_history = []
    
    steps_per_epoch = len(dataset) // Settings.BATCH_SIZE
    
    for epoch in range(Settings.EPOCHS):
        total_loss = 0
        beta = Settings.PER_BETA_START + (1.0 - Settings.PER_BETA_START) * (epoch / Settings.EPOCHS)
        
        pbar = tqdm(range(steps_per_epoch), desc=f"Epoch {epoch+1}/{Settings.EPOCHS}")
        
        for _ in pbar:
            # SAMPLE FROM PER
            # Unpack hack: actions=next_states, rewards=curr_prices, next_states=next_prices
            states, next_states_hack, curr_prices_hack, next_prices_hack, _, idxs, is_weights = memory.sample(Settings.BATCH_SIZE, beta)
            
            state_tensor = torch.FloatTensor(states).to(device)
            next_state_tensor = torch.FloatTensor(next_states_hack).to(device)
            curr_price = torch.FloatTensor(curr_prices_hack).to(device)
            next_price = torch.FloatTensor(next_prices_hack).to(device)
            weights_tensor = torch.FloatTensor(is_weights).to(device)
            
            batch_size = state_tensor.size(0)
            
            # Epsilon Greedy
            if random.random() < epsilon:
                action_tensor = torch.randint(0, Settings.OUTPUT_DIM, (batch_size,), device=device)
            else:
                with torch.no_grad():
                    q_values = policy_net(state_tensor)
                    action_tensor = torch.argmax(q_values, dim=1)
            
            # Reward Calculation
            price_diff = next_price - curr_price
            norm_pnl = price_diff * SCALING_FACTOR
            total_cost = (SPREAD + COMMISSION) * SCALING_FACTOR
            base_penalty = -0.1
            reward_tensor = torch.full((batch_size,), base_penalty, device=device)
            
            is_buy = (action_tensor == 1)
            is_sell = (action_tensor == 2)
            
            reward_tensor[is_buy] = torch.tanh(norm_pnl - total_cost)[is_buy]
            reward_tensor[is_sell] = torch.tanh(-norm_pnl - total_cost)[is_sell]
            
            # Update
            current_q = policy_net(state_tensor).gather(1, action_tensor.unsqueeze(1)).squeeze(1)
            with torch.no_grad():
                next_q = policy_net(next_state_tensor).max(1)[0]
                target_q = reward_tensor + (Settings.GAMMA * next_q)
                
            loss_element = loss_fn(current_q, target_q)
            loss = (loss_element * weights_tensor).mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Update Priorities
            td_errors = torch.abs(target_q - current_q).detach().cpu().numpy()
            memory.update_priorities(idxs, td_errors)
            
            total_loss += loss.item()
            
        if epsilon > Settings.EPSILON_MIN:
            epsilon *= Settings.EPSILON_DECAY
            
        avg_loss = total_loss / steps_per_epoch
        loss_history.append(avg_loss)
        print(f"Avg Loss: {avg_loss:.6f}, Epsilon: {epsilon:.4f}")
        
    drive_save_path = os.path.join(os.path.dirname(csv_path), f"{symbol}_brain.pth")
    torch.save(policy_net.state_dict(), drive_save_path)
    print(f"Model Saved to {drive_save_path}!")
    
    plt.plot(loss_history)
    plt.title(f"Training Loss - {symbol}")
    plt.show()

In [ ]:
# --- EXECUTION ---
drive_data_path = "/content/drive/MyDrive/data"

# ⚡ SELECT PAIRS TO TRAIN HERE:
pairs = ['EURUSD'] # Change to ['XAUUSD'] or Settings.PAIRS for all

print(f"Training Pairs: {pairs}")

for symbol in pairs:
    csv_path = os.path.join(drive_data_path, f"{symbol}.csv")
    if os.path.exists(csv_path):
        print(f"Found data for {symbol} at: {csv_path}")
        train_model(symbol, csv_path)
    else:
        print(f"Data for {symbol} not found. Skipping.")


In [ ]:
# --- BACKTEST ENGINE (Colab Version - Optimized) ---

def colab_backtest(symbol, csv_path):
    print(f"\n--- Starting Backtest for {symbol} ---")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model_path = os.path.join(os.path.dirname(csv_path), f"{symbol}_brain.pth")
    
    if not os.path.exists(model_path):
        print(f"Model {model_path} not found. Please train first.")
        return

    policy_net = QNetwork().to(device)
    try:
        policy_net.load_state_dict(torch.load(model_path, map_location=device))
    except RuntimeError as e:
        print(f"\n!!! MODEL ARCHITECTURE MISMATCH !!!")
        print(f"Error: {e}")
        print(f"You need to RETRAIN the model because FEATURES have changed.")
        return
        
    policy_net.eval()
    print("Model loaded.")
    
    df = pd.read_csv(csv_path, parse_dates=['time'], index_col='time')
    test_start_idx = Settings.TRAIN_SPLIT_INDEX 
    if len(df) < test_start_idx + 100:
        print(f"Not enough data for backtest. Total: {len(df)}, Needed: >{test_start_idx}")
        return
        
    print(f"Backtesting on rows {test_start_idx} to {len(df)}...")
    
    df = prepare_features(df)
    
    df_test = df.iloc[test_start_idx:].copy()
    
    feature_data = df_test[Settings.FEATURES].values
    opens = df_test['open'].values
    highs = df_test['high'].values
    lows = df_test['low'].values
    closes = df_test['close'].values
    times = df_test.index
    from pandas_ta import atr
    # Re-calculate ATR here to be safe for Stop Loss
    _atr = ta.atr(df_test['high'], df_test['low'], df_test['close'], length=14)
    _atr.fillna(0, inplace=True)
    atrs = _atr.values

    balance = 10000.0
    equity_curve = [balance]
    position = 0 
    entry_price = 0.0
    stop_loss = 0.0
    
    contract_size = 100
    spread_cost = 0.20
    txn_cost = 0.0
    lot_size = 0.01
    
    if symbol in Settings.PAIR_CONFIGS:
         profile = Settings.PAIR_CONFIGS[symbol]
         contract_size = profile['contract_size']
         spread_cost = profile['spread'] * contract_size * lot_size
         txn_cost = profile['commission'] * contract_size * lot_size
    
    trades = []
    
    for t in tqdm(range(Settings.SEQUENCE_LENGTH, len(df_test) - 1)):
        state_tensor = torch.FloatTensor(feature_data[t - Settings.SEQUENCE_LENGTH : t]).unsqueeze(0).to(device)
        
        with torch.no_grad():
            q = policy_net(state_tensor)
            action = torch.argmax(q, dim=1).item()
            
            if action != 0: 
                 # --- FILTERS ---
                 current_time = times[t]
                 
                 # 1. ADX FILTER (Deep Market Avoidance)
                 current_adx = state_tensor[0, -1, -1].item() * 100.0 
                 if current_adx < 25:
                     action = 0 
                 
                 # 2. BOLLINGER SQUEEZE FILTER
                 prev_time = times[t-1]
                 try:
                     bb_u = df_test.loc[prev_time, 'BBU_20_2.0']
                     bb_l = df_test.loc[prev_time, 'BBL_20_2.0']
                     prev_close = df_test.loc[prev_time, 'close']
                         
                     width = bb_u - bb_l
                     vol_pct = width / prev_close
                     if vol_pct < 0.0005:
                          action = 0
                 except KeyError:
                     pass
            
        # Execution
        next_open = opens[t]
        next_high = highs[t]
        next_low = lows[t]
        next_time = times[t]
        atr = atrs[t-1] 
        
        trade_closed = False
        exit_price = 0.0
        
        if position != 0:
            if next_time.hour >= 20 and next_time.minute == 0:
                exit_price = next_open
                trade_closed = True
            elif position == 1:
                 if next_low <= stop_loss:
                      exit_price = stop_loss
                      trade_closed = True
                 elif (next_high - entry_price) > (1.0 * atr) and stop_loss < entry_price:
                      stop_loss = entry_price
            elif position == -1:
                 if next_high >= stop_loss:
                      exit_price = stop_loss
                      trade_closed = True
                 elif (entry_price - next_low) > (1.0 * atr) and stop_loss > entry_price:
                      stop_loss = entry_price
                
            if trade_closed:
                if position == 1:
                     gross_pnl = (exit_price - entry_price) * contract_size * lot_size
                else:
                     gross_pnl = (entry_price - exit_price) * contract_size * lot_size
                     
                net_pnl = gross_pnl - txn_cost
                balance += net_pnl
                trades.append(net_pnl)
                position = 0
        
        if not trade_closed:
            if action == 1: 
                if position == -1: 
                     exit_price = next_open
                     gross_pnl = (entry_price - exit_price) * contract_size * lot_size
                     net_pnl = gross_pnl - txn_cost
                     balance += net_pnl
                     trades.append(net_pnl)
                     position = 1
                     entry_price = next_open
                     stop_loss = entry_price - (atr * 2.5)
                elif position == 0:
                     position = 1
                     entry_price = next_open
                     stop_loss = entry_price - (atr * 2.5)
            elif action == 2: 
                if position == 1: 
                     exit_price = next_open
                     gross_pnl = (exit_price - entry_price) * contract_size * lot_size
                     net_pnl = gross_pnl - txn_cost
                     balance += net_pnl
                     trades.append(net_pnl)
                     position = -1
                     entry_price = next_open
                     stop_loss = entry_price + (atr * 2.5)
                elif position == 0:
                     position = -1
                     entry_price = next_open
                     stop_loss = entry_price + (atr * 2.5)
                     
        equity_curve.append(balance)
        
    win_rate = sum(1 for t in trades if t > 0) / len(trades) * 100 if trades else 0
    print(f"\nFinal Balance: ${balance:.2f}")
    print(f"Total Trades: {len(trades)}")
    print(f"Win Rate: {win_rate:.1f}%")
    
    plt.figure(figsize=(12,6))
    plt.plot(equity_curve)
    plt.title(f"Backtest Equity - {symbol}")
    plt.show()

for symbol in Settings.PAIRS:
     csv_path = os.path.join(drive_data_path, f"{symbol}.csv")
     if os.path.exists(csv_path):
         colab_backtest(symbol, csv_path)